dans ce fichier nous allons mettre les requêtes de filtres et nettoyage des données

In [2]:
import pandas as pd

In [3]:
films_tmdb_imdb = pd.read_csv("./donnees/table_films_select.csv", sep="\t", low_memory=False)

In [4]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770236 entries, 0 to 770235
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            770236 non-null  int64  
 1   adult                 770236 non-null  bool   
 2   budget                770236 non-null  int64  
 3   genres                770236 non-null  object 
 4   id                    770236 non-null  int64  
 5   imdb_id               770236 non-null  object 
 6   original_language     770236 non-null  object 
 7   original_title        770236 non-null  object 
 8   overview              732422 non-null  object 
 9   popularity            770236 non-null  float64
 10  poster_path           729654 non-null  object 
 11  production_countries  770236 non-null  object 
 12  release_date          762336 non-null  object 
 13  revenue               770236 non-null  int64  
 14  runtime               770236 non-null  int64  
 15  

In [5]:
films_tmdb_imdb = films_tmdb_imdb.drop('Unnamed: 0', axis = 1)

In [6]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770236 entries, 0 to 770235
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   adult                 770236 non-null  bool   
 1   budget                770236 non-null  int64  
 2   genres                770236 non-null  object 
 3   id                    770236 non-null  int64  
 4   imdb_id               770236 non-null  object 
 5   original_language     770236 non-null  object 
 6   original_title        770236 non-null  object 
 7   overview              732422 non-null  object 
 8   popularity            770236 non-null  float64
 9   poster_path           729654 non-null  object 
 10  production_countries  770236 non-null  object 
 11  release_date          762336 non-null  object 
 12  revenue               770236 non-null  int64  
 13  runtime               770236 non-null  int64  
 14  spoken_languages      770236 non-null  object 
 15  

In [7]:
films_tmdb_imdb['release_date']=pd.to_datetime(films_tmdb_imdb['release_date'])

In [8]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770236 entries, 0 to 770235
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 770236 non-null  bool          
 1   budget                770236 non-null  int64         
 2   genres                770236 non-null  object        
 3   id                    770236 non-null  int64         
 4   imdb_id               770236 non-null  object        
 5   original_language     770236 non-null  object        
 6   original_title        770236 non-null  object        
 7   overview              732422 non-null  object        
 8   popularity            770236 non-null  float64       
 9   poster_path           729654 non-null  object        
 10  production_countries  770236 non-null  object        
 11  release_date          762336 non-null  datetime64[ns]
 12  revenue               770236 non-null  int64         
 13 

In [ ]:
On drop tagline car rarement l'info

In [9]:
films_tmdb_imdb = films_tmdb_imdb.drop('tagline',axis=1)

On va garder que ce qui ont un résumé
===>>> supprime les lignes avec une valeur null

In [10]:
films_tmdb_imdb.dropna(subset = ['overview'], inplace=True)

In [11]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 732422 entries, 0 to 770235
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 732422 non-null  bool          
 1   budget                732422 non-null  int64         
 2   genres                732422 non-null  object        
 3   id                    732422 non-null  int64         
 4   imdb_id               732422 non-null  object        
 5   original_language     732422 non-null  object        
 6   original_title        732422 non-null  object        
 7   overview              732422 non-null  object        
 8   popularity            732422 non-null  float64       
 9   poster_path           697788 non-null  object        
 10  production_countries  732422 non-null  object        
 11  release_date          725935 non-null  datetime64[ns]
 12  revenue               732422 non-null  int64         
 13  runt

In [13]:
films_tmdb_imdb.dropna(subset = ['poster_path'], inplace=True)

In [14]:
films_tmdb_imdb.isna().sum()

adult                      0
budget                     0
genres                     0
id                         0
imdb_id                    0
original_language          0
original_title             0
overview                   0
popularity                 0
poster_path                0
production_countries       0
release_date            3016
revenue                    0
runtime                    0
spoken_languages           0
status                     0
title                      0
vote_average               0
vote_count                 0
tconst                     0
directors                  0
writers                    0
nconst                    75
category                  75
characters                75
primaryName               75
knownForTitles            75
dtype: int64

On garde que si info sur personne = 
nconst                     
category                   
characters                 
primaryName                
knownForTitles

In [15]:
films_tmdb_imdb.dropna(subset = ['nconst', 'category', 'characters','primaryName', 'knownForTitles'], inplace=True)

In [16]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 697713 entries, 0 to 770235
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 697713 non-null  bool          
 1   budget                697713 non-null  int64         
 2   genres                697713 non-null  object        
 3   id                    697713 non-null  int64         
 4   imdb_id               697713 non-null  object        
 5   original_language     697713 non-null  object        
 6   original_title        697713 non-null  object        
 7   overview              697713 non-null  object        
 8   popularity            697713 non-null  float64       
 9   poster_path           697713 non-null  object        
 10  production_countries  697713 non-null  object        
 11  release_date          694699 non-null  datetime64[ns]
 12  revenue               697713 non-null  int64         
 13  runt

In [17]:

films_tmdb_imdb = films_tmdb_imdb.drop('tconst', axis =1)

On renomme id en id_tmdb pour calculer

In [18]:
films_tmdb_imdb = films_tmdb_imdb.rename(columns = {'id' : 'id_tmdb'})

In [19]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 697713 entries, 0 to 770235
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 697713 non-null  bool          
 1   budget                697713 non-null  int64         
 2   genres                697713 non-null  object        
 3   id_tmdb               697713 non-null  int64         
 4   imdb_id               697713 non-null  object        
 5   original_language     697713 non-null  object        
 6   original_title        697713 non-null  object        
 7   overview              697713 non-null  object        
 8   popularity            697713 non-null  float64       
 9   poster_path           697713 non-null  object        
 10  production_countries  697713 non-null  object        
 11  release_date          694699 non-null  datetime64[ns]
 12  revenue               697713 non-null  int64         
 13  runt

In [20]:
films_tmdb_imdb = films_tmdb_imdb.rename(columns={'id':'id_tmdb'})

In [21]:
films_tmdb_imdb[['vote_average', 'popularity']].value_counts(sort = True)

vote_average  popularity
0.0           0.600         26781
              1.400          3840
              0.840          3446
5.0           0.600          3440
6.0           0.600          3332
                            ...  
5.1           0.865             1
5.4           0.727             1
4.9           1.449             1
4.8           0.924             1
5.8           1.118             1
Name: count, Length: 29489, dtype: int64

In [22]:
films_tmdb_imdb['vote_average'].max()

10.0

On prend les films avec vote_average supérieur à 5

In [24]:
films_tmdb_imdb_sup5 = films_tmdb_imdb[films_tmdb_imdb['vote_average']>5]

In [25]:
films_tmdb_imdb_sup5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 463029 entries, 0 to 770134
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 463029 non-null  bool          
 1   budget                463029 non-null  int64         
 2   genres                463029 non-null  object        
 3   id_tmdb               463029 non-null  int64         
 4   imdb_id               463029 non-null  object        
 5   original_language     463029 non-null  object        
 6   original_title        463029 non-null  object        
 7   overview              463029 non-null  object        
 8   popularity            463029 non-null  float64       
 9   poster_path           463029 non-null  object        
 10  production_countries  463029 non-null  object        
 11  release_date          462885 non-null  datetime64[ns]
 12  revenue               463029 non-null  int64         
 13  runt

films selon la date de sortie

In [26]:
from datetime import datetime

In [27]:
films_tmdb_imdb_sup5['year'] = films_tmdb_imdb_sup5['release_date'].dt.year

C:\Users\filiz\AppData\Local\Temp\ipykernel_31720\2621779178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films_tmdb_imdb_sup5['year'] = films_tmdb_imdb_sup5['release_date'].dt.year


In [28]:
films_tmdb_imdb_sup5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 463029 entries, 0 to 770134
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 463029 non-null  bool          
 1   budget                463029 non-null  int64         
 2   genres                463029 non-null  object        
 3   id_tmdb               463029 non-null  int64         
 4   imdb_id               463029 non-null  object        
 5   original_language     463029 non-null  object        
 6   original_title        463029 non-null  object        
 7   overview              463029 non-null  object        
 8   popularity            463029 non-null  float64       
 9   poster_path           463029 non-null  object        
 10  production_countries  463029 non-null  object        
 11  release_date          462885 non-null  datetime64[ns]
 12  revenue               463029 non-null  int64         
 13  runt

In [29]:
films_tmdb_imdb_sup5_2000 = films_tmdb_imdb_sup5[films_tmdb_imdb_sup5['year']> 1999]

In [30]:
films_tmdb_imdb_sup5_2000.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272237 entries, 70 to 770046
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 272237 non-null  bool          
 1   budget                272237 non-null  int64         
 2   genres                272237 non-null  object        
 3   id_tmdb               272237 non-null  int64         
 4   imdb_id               272237 non-null  object        
 5   original_language     272237 non-null  object        
 6   original_title        272237 non-null  object        
 7   overview              272237 non-null  object        
 8   popularity            272237 non-null  float64       
 9   poster_path           272237 non-null  object        
 10  production_countries  272237 non-null  object        
 11  release_date          272237 non-null  datetime64[ns]
 12  revenue               272237 non-null  int64         
 13  run

In [33]:
films_tmdb_imdb_sup5['year'].value_counts(sort=True)

year
2019.0    17487
2017.0    17399
2022.0    16683
2018.0    16511
2016.0    15588
          ...  
1898.0       20
1894.0        8
1892.0        7
1874.0        4
1878.0        1
Name: count, Length: 133, dtype: int64

On va prendre les films après 2000

In [34]:
films_tmdb_imdb_sup5_2000 = films_tmdb_imdb_sup5[films_tmdb_imdb_sup5['year']>1999]

In [31]:
films_tmdb_imdb_sup5_2000.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272237 entries, 70 to 770046
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 272237 non-null  bool          
 1   budget                272237 non-null  int64         
 2   genres                272237 non-null  object        
 3   id_tmdb               272237 non-null  int64         
 4   imdb_id               272237 non-null  object        
 5   original_language     272237 non-null  object        
 6   original_title        272237 non-null  object        
 7   overview              272237 non-null  object        
 8   popularity            272237 non-null  float64       
 9   poster_path           272237 non-null  object        
 10  production_countries  272237 non-null  object        
 11  release_date          272237 non-null  datetime64[ns]
 12  revenue               272237 non-null  int64         
 13  run

In [32]:
len(films_tmdb_imdb_sup5_2000['id_tmdb'].unique())

14672

In [33]:
liste = list(films_tmdb_imdb_sup5_2000['runtime'].unique())
liste.sort()
liste

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [34]:
films_tmdb_imdb_sup5_2000_60 = films_tmdb_imdb_sup5_2000[films_tmdb_imdb_sup5_2000['runtime']>=60]

In [35]:
films_tmdb_imdb_sup5_2000_60.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246079 entries, 70 to 769918
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 246079 non-null  bool          
 1   budget                246079 non-null  int64         
 2   genres                246079 non-null  object        
 3   id_tmdb               246079 non-null  int64         
 4   imdb_id               246079 non-null  object        
 5   original_language     246079 non-null  object        
 6   original_title        246079 non-null  object        
 7   overview              246079 non-null  object        
 8   popularity            246079 non-null  float64       
 9   poster_path           246079 non-null  object        
 10  production_countries  246079 non-null  object        
 11  release_date          246079 non-null  datetime64[ns]
 12  revenue               246079 non-null  int64         
 13  run

Nombre de films qui durent plus de 60 minutes inclues sortis à partir de 2000 et qui ont une note supérieure à 5.

In [36]:
len(films_tmdb_imdb_sup5_2000_60['id_tmdb'].unique())

12771

On va regarder les langues parlées dans les films

In [37]:
films_tmdb_imdb_sup5_2000_60 ['spoken_languages'].unique()

array(["['en']", "['en', 'ja', 'fr']", "['en', 'tr', 'el', 'zh']", ...,
       "['fr', 'en', 'tr']", "['ky', 'ru']", "['fr', 'wo', 'it', 'en']"],
      dtype=object)

On prend les films qui sont sortis(Released)et on enlève les films In Production

In [38]:
films_tmdb_imdb_sup5_2000_60_released = films_tmdb_imdb_sup5_2000_60[films_tmdb_imdb_sup5_2000_60['status'] =='In Production']

In [39]:
films_tmdb_imdb_sup5_2000_60_released = films_tmdb_imdb_sup5_2000_60[films_tmdb_imdb_sup5_2000_60['status'] =='Released']

In [40]:
films_tmdb_imdb_sup5_2000_60_released.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246079 entries, 70 to 769918
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 246079 non-null  bool          
 1   budget                246079 non-null  int64         
 2   genres                246079 non-null  object        
 3   id_tmdb               246079 non-null  int64         
 4   imdb_id               246079 non-null  object        
 5   original_language     246079 non-null  object        
 6   original_title        246079 non-null  object        
 7   overview              246079 non-null  object        
 8   popularity            246079 non-null  float64       
 9   poster_path           246079 non-null  object        
 10  production_countries  246079 non-null  object        
 11  release_date          246079 non-null  datetime64[ns]
 12  revenue               246079 non-null  int64         
 13  run

In [41]:
len(films_tmdb_imdb_sup5_2000_60_released['original_title'].unique())

12644

In [42]:
len(films_tmdb_imdb_sup5_2000_60_released['imdb_id'].unique())

12771

In [44]:
films_tmdb_imdb_sup5_2000_60_released['original_title'].value_counts()

original_title
Home                                             61
Deux                                             61
Vendetta                                         59
Anna                                             56
Charlie's Angels                                 53
                                                 ..
Antifa: Chasseurs de skins                        1
La Vallée close                                   1
Musk vs Bezos, la nouvelle guerre des étoiles     1
Des traîtres dans la Résistance                   1
Ta'ang                                            1
Name: count, Length: 12644, dtype: int64

In [45]:
films_tmdb_imdb_sup5_2000_60_released[films_tmdb_imdb_sup5_2000_60_released['original_title']=="Charlie's Angels"]

,adult,budget,genres,id_tmdb,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,directors,writers,nconst,category,characters,primaryName,knownForTitles,year
16107,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000139,actress,"[""Natalie""]",Cameron Diaz,"tt0259711,tt0129387,tt0160127,tt0119738",2000.0
16108,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000106,actress,"[""Dylan""]",Drew Barrymore,"tt0160127,tt0151738,tt0343660,tt0120631",2000.0
16109,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0005154,actress,"[""Alex""]",Lucy Liu,"tt0266697,tt0160127,tt0299658,tt0305357",2000.0
16110,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000195,actor,"[""Bosley""]",Bill Murray,"tt0335266,tt0128445,tt1748122,tt0362270",2000.0
16111,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0005377,actor,"[""Eric Knox""]",Sam Rockwell,"tt1182345,tt5027774,tt0270288,tt1931533",2000.0
16112,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0001488,actress,"[""Vivian Wood""]",Kelly Lynch,"tt0097240,tt0160127,tt0098206,tt0114857",2000.0
16113,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000347,actor,"[""Roger Corwin""]",Tim Curry,"tt0112715,tt0088930,tt0117110,tt0073629",2000.0
16114,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000417,actor,"[""Thin Man""]",Crispin Glover,"tt0118141,tt0310357,tt0091860,tt7972570",2000.0
16115,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0001455,actor,"[""Jason""]",Matt LeBlanc,"tt0108778,tt0120738,tt1582350,tt0375355",2000.0
16116,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0005112,actor,"[""Mr. Jones""]",LL Cool J,"tt0149261,tt0297284,tt0120694,tt1378167",2000.0


In [47]:
films_tmdb_imdb_sup5_2000_60_released[films_tmdb_imdb_sup5_2000_60_released['original_title']=="Charlie's Angels"]

,adult,budget,genres,id_tmdb,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,directors,writers,nconst,category,characters,primaryName,knownForTitles,year
16107,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000139,actress,"[""Natalie""]",Cameron Diaz,"tt0259711,tt0129387,tt0160127,tt0119738",2000.0
16108,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000106,actress,"[""Dylan""]",Drew Barrymore,"tt0160127,tt0151738,tt0343660,tt0120631",2000.0
16109,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0005154,actress,"[""Alex""]",Lucy Liu,"tt0266697,tt0160127,tt0299658,tt0305357",2000.0
16110,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000195,actor,"[""Bosley""]",Bill Murray,"tt0335266,tt0128445,tt1748122,tt0362270",2000.0
16111,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0005377,actor,"[""Eric Knox""]",Sam Rockwell,"tt1182345,tt5027774,tt0270288,tt1931533",2000.0
16112,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0001488,actress,"[""Vivian Wood""]",Kelly Lynch,"tt0097240,tt0160127,tt0098206,tt0114857",2000.0
16113,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000347,actor,"[""Roger Corwin""]",Tim Curry,"tt0112715,tt0088930,tt0117110,tt0073629",2000.0
16114,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0000417,actor,"[""Thin Man""]",Crispin Glover,"tt0118141,tt0310357,tt0091860,tt7972570",2000.0
16115,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0001455,actor,"[""Jason""]",Matt LeBlanc,"tt0108778,tt0120738,tt1582350,tt0375355",2000.0
16116,False,92000000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Th...",4327,tt0160127,en,Charlie's Angels,The captivating crime-fighting trio who are ma...,23.703,/u5GbILxLgfeNHCnTtTRuhmAC9i4.jpg,...,5.818,3751,nm0629334,"nm0746598,nm0004412,nm0041864,nm0324578,nm0730850",nm0005112,actor,"[""Mr. Jones""]",LL Cool J,"tt0149261,tt0297284,tt0120694,tt1378167",2000.0


In [48]:
len(test_films['id_tmdb'].unique())

NameError: name 'test_films' is not defined

In [ ]:
len(test_films['title'].unique())

On regarde dans title_akas la région de la version du titre et on prend quand France
On met na=False pour ignorer les valeurs NaN sinon on a une erreur

In [50]:
#title_akas_FR = title_akas[title_akas['region'].str.contains('FR', na=False)]

In [ ]:
#title_akas_FR.info()

In [ ]:
#title_akas_FR_reduit = title_akas_FR[['titleId','title','region','language']]
# title_akas_FR_reduit.info()

In [ ]:
# films_tmdb_imdb_sup5_2000_60_released_regFR = pd.merge(films_tmdb_imdb_sup5_2000_60_released,
#                                                        title_akas_FR_reduit,
#                                                        how='inner',
#                                                        left_on='imdb_id',
#                                                        right_on='titleId')

In [51]:
films_select = films_tmdb_imdb_sup5_2000_60_released[['adult','budget','genres','id_tmdb','imdb_id','original_language','original_title','original_title','overview','popularity','poster_path','production_countries','release_date' ,'revenue','runtime','spoken_languages','status','title' ,'vote_average','vote_count','directors','writers','year'                        

SyntaxError: incomplete input (501058827.py, line 1)

In [52]:
films_select.duplicated().sum()

NameError: name 'films_select' is not defined

In [ ]:
films_select = films_select.drop_duplicates()

In [ ]:
films_select.info()

In [ ]:
films_select.to_csv('./donnees/films_selectionnes.csv', sep="\t")

In [ ]:
films_tmdb_imdb_sup5_2000_60_released.to_csv('./donnees/films_tmdb_imdb_sup5_2000_60_released.csv', sep="\t")

In [ ]:
films = pd.read_csv("./donnees/films_selectionnes.csv", sep="\t", low_memory=False)

In [ ]:
title_basics.head()

In [ ]:
title_basics.duplicated().sum()

In [ ]:
title_basics['tconst'].value_counts()

In [ ]:
title_basics['tconst'].duplicated().sum()

In [ ]:
films.duplicated().sum()

In [ ]:
films['imdb_id'].duplicated().sum()

In [ ]:
films.head()

In [ ]:
verif_que_films = pd.merge(films,
                           title_basics,
                           how='left',
                           left_on='imdb_id',
                           right_on = 'tconst')

verif_que_films

In [ ]:
verif_que_films['titleType'].value_counts()

In [ ]:
verif_que_films[['runtime','runtimeMinutes']]

In [ ]:
verif_que_films.columns

In [ ]:
films_v2 = verif_que_films[verif_que_films['titleType'] == 'movie']
films_v2.head()

In [ ]:
films_v2 = films_v2.drop(['tconst','titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_y','original_title.1'], axis=1)

In [ ]:
films_personnes_import = pd.read_csv('./donnees/films_tmdb_imdb_sup5_2000_60_released.csv', sep="\t")

In [ ]:
liste_code_tmdb = list(films_v2['id_tmdb'])
liste_code_tmdb

In [ ]:
films_personnes = films_personnes_import [films_personnes_import['id_tmdb'].isin(liste_code_tmdb)]
films_personnes.head()

In [ ]:
films_v2.info()

In [ ]:
films_personnes.info()

In [ ]:
films.columns

In [ ]:
films_personnes.columns

In [ ]:
from datetime import datetime

In [ ]:
films_v2['release_date'] = pd.to_datetime(films_v2['release_date'])

In [ ]:
films_v2['year'] = films_v2['release_date'].dt.year
films_v2['year']

In [ ]:
films_v2.info()

In [ ]:
max_year = films_v2['year'].max()
max_year

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(films_v2, x='budget')

In [ ]:
sns.boxplot(films_v2, x='revenue');

In [ ]:
sns.boxplot(films_v2, x='runtime');

In [ ]:
films_v2['runtime'].max()

In [ ]:
films_v2[(films_v2['runtime'] > 240)]

In [ ]:
sns.boxplot(films_v2, x='vote_average');

In [ ]:
sns.boxplot(films_v2, x='vote_count');

In [ ]:
films_v2.to_csv('./donnees/export_films.csv', sep="\t")

In [ ]:

films_personnes.to_csv('./donnees/export_films_personnes.csv', sep="\t")